In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

# Alternatives matrix and data
matrix = np.array([
    [0.081, 0.075, 0.105, 0.101],
    [0.058, 0.073, 0.087, 0.093],
    [0.052, 0.087, 0.073, 0.09],
    [0.039, 0.079, 0.081, 0.08],
    [0.080, 0.077, 0.239, 0.071],
    [0.278, 0.058, 0.117, 0.062],
    [0.038, 0.123, 0.015, 0.131],
    [0.028, 0.089, 0.018, 0.134],
    [0.036, 0.119, 0.108, 0.095],
    [0.098, 0.093, 0.121, 0.081],
    [0.013, 0.127, 0.038, 0.063]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10', 'Site11']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

# Normalize matrix
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

norm_matrix = normalize_matrix(matrix, criteria_types)
df_norm = pd.DataFrame(norm_matrix, columns=criteria, index=sites)

# Criterion-wise rankings using scipy
print("\nRanking of sites for each criterion:")
criterion_ranks = pd.DataFrame(index=sites)

for i, c in enumerate(criteria):
    col = norm_matrix[:, i]
    if criteria_types[i] == 'max':
        ranks = rankdata(-col, method='min')  # Descending rank
    else:
        ranks = rankdata(col, method='min')   # Ascending rank
    criterion_ranks[c] = ranks

print(criterion_ranks)

# AHP weighted score and overall ranking
overall_scores = np.dot(norm_matrix, weights)
overall_ranks = rankdata(-overall_scores, method='min')  # Higher score = better

df_result = pd.DataFrame({
    'AHP Score': overall_scores,
    'Rank': overall_ranks
}, index=sites).sort_values('Rank')

print("\nOverall AHP Scores and Ranking:")
print(df_result)



Ranking of sites for each criterion:
        C1  C2  C3  C4
Site1    3   9   5   3
Site2    5  10   6   5
Site3    6   6   8   6
Site4    7   7   7   8
Site5    4   8   1   9
Site6    1  11   3  11
Site7    8   2  11   2
Site8   10   5  10   1
Site9    9   3   4   4
Site10   2   4   2   7
Site11  11   1   9  10

Overall AHP Scores and Ranking:
        AHP Score  Rank
Site5    0.596394     1
Site9    0.503506     2
Site10   0.442362     3
Site6    0.375018     4
Site7    0.367411     5
Site11   0.344750     6
Site1    0.342667     7
Site3    0.298308     8
Site2    0.274119     9
Site4    0.261928    10
Site8    0.224740    11
